In [1]:
!pip install opencv-python mediapipe pycaw numpy


     ---------------------------------------- 0.0/104.0 kB ? eta -:--:--
     -------------------------------------- 104.0/104.0 kB 2.0 MB/s eta 0:00:00
   ---------------------------------------- 0.0/51.0 MB ? eta -:--:--
   ---------------------------------------- 0.2/51.0 MB 5.0 MB/s eta 0:00:11
   ---------------------------------------- 0.4/51.0 MB 3.8 MB/s eta 0:00:14
   ---------------------------------------- 0.4/51.0 MB 3.5 MB/s eta 0:00:15
   ---------------------------------------- 0.6/51.0 MB 2.9 MB/s eta 0:00:18
   ---------------------------------------- 0.6/51.0 MB 2.7 MB/s eta 0:00:19
    --------------------------------------- 0.7/51.0 MB 2.7 MB/s eta 0:00:19
    --------------------------------------- 0.8/51.0 MB 2.6 MB/s eta 0:00:20
    --------------------------------------- 0.9/51.0 MB 2.5 MB/s eta 0:00:20
    --------------------------------------- 1.0/51.0 MB 2.4 MB/s eta 0:00:22
    --------------------------------------- 1.1/51.0 MB 2.4 MB/s eta 0:00:21
    ---


[notice] A new release of pip is available: 23.3.2 -> 25.0
[notice] To update, run: python.exe -m pip install --upgrade pip


   ---------------------- ----------------- 36.6/63.9 MB 1.8 MB/s eta 0:00:15
   ---------------------- ----------------- 36.7/63.9 MB 1.8 MB/s eta 0:00:15
   ----------------------- ---------------- 36.9/63.9 MB 1.8 MB/s eta 0:00:15
   ----------------------- ---------------- 37.0/63.9 MB 1.8 MB/s eta 0:00:15
   ----------------------- ---------------- 37.0/63.9 MB 1.9 MB/s eta 0:00:15
   ----------------------- ---------------- 37.1/63.9 MB 1.9 MB/s eta 0:00:15
   ----------------------- ---------------- 37.2/63.9 MB 1.8 MB/s eta 0:00:15
   ----------------------- ---------------- 37.3/63.9 MB 1.8 MB/s eta 0:00:15
   ----------------------- ---------------- 37.4/63.9 MB 1.8 MB/s eta 0:00:15
   ----------------------- ---------------- 37.5/63.9 MB 1.8 MB/s eta 0:00:15
   ----------------------- ---------------- 37.6/63.9 MB 1.8 MB/s eta 0:00:15
   ----------------------- ---------------- 37.6/63.9 MB 1.8 MB/s eta 0:00:15
   ----------------------- ---------------- 37.7/63.9 MB 1.8 MB/

In [18]:
import cv2
import mediapipe as mp
import numpy as np
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
from comtypes import CLSCTX_ALL
import time
import math


In [25]:
wCam,hCam=640,480

mpHands = mp.solutions.hands
hands = mpHands.Hands(max_num_hands=2)  # Detect up to 2 hands
mpDraw = mp.solutions.drawing_utils

cap=cv2.VideoCapture(0)
cap.set(3,wCam)
cap.set(4,hCam)
pTime=0
THUMB_TIP = 4
INDEX_TIP = 8

devices = AudioUtilities.GetSpeakers()
interface = devices.Activate(
    IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
volume = interface.QueryInterface(IAudioEndpointVolume)
print(volume.GetVolumeRange())
#volRange=volume.GetVolumeRange()
volume.SetMasterVolumeLevel(0,None)
minVol=volRange[0]
maxVol=volRange[1]
while True:
    success,img=cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = hands.process(imgRGB) 

    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)  # Draw hand landmarks

            # Extract landmark positions for thumb and index finger
            h, w, c = img.shape  # Get image dimensions
            thumb_x, thumb_y = None, None
            index_x, index_y = None, None

            for id, lm in enumerate(handLms.landmark):
                cx, cy = int(lm.x * w), int(lm.y * h)  # Convert normalized coordinates to pixels
                
                if id == THUMB_TIP:  # Thumb tip
                    thumb_x, thumb_y = cx, cy
                    cv2.circle(img, (cx, cy), 10, (0, 255, 0), cv2.FILLED)  # Green circle on thumb

                if id == INDEX_TIP:  # Index finger tip
                    index_x, index_y = cx, cy
                    cv2.circle(img, (cx, cy), 10, (255, 0, 0), cv2.FILLED)  # Blue circle on index finger

            # Display thumb & index tip coordinates on the screen
            if thumb_x is not None and index_x is not None:
                cv2.line(img, (thumb_x, thumb_y), (index_x, index_y), (0, 255, 255), 3)
                mid_x, mid_y = (thumb_x + index_x) // 2, (thumb_y + index_y) // 2
                cv2.circle(img, (mid_x, mid_y), 10, (0, 0, 255), cv2.FILLED)
                length=math.hypot(index_x-thumb_x,index_y-thumb_y)
                #print(length)
                vol=np.interp(length,[50,300],[minVol,maxVol])
                print(vol)
                volume.SetMasterVolumeLevel(vol,None)
                if length<20:
                    cv2.circle(img,(mid_x,mid_y),15,(0,255,0),cv2.FILLED)
                cv2.putText(img, f"Thumb: ({thumb_x}, {thumb_y})", (40, 100), cv2.FONT_HERSHEY_COMPLEX, 0.8, (0, 255, 0), 2)
                cv2.putText(img, f"Index: ({index_x}, {index_y})", (40, 140), cv2.FONT_HERSHEY_COMPLEX, 0.8, (255, 0, 0), 2)


    cTime=time.time()
    fps=1/(cTime-pTime)
    pTime=cTime

    cv2.putText(img,f"FPS :{int(fps)}",(40,50),cv2.FONT_HERSHEY_COMPLEX,1,(255,0,0),3)
    cv2.imshow("img",img)
    cv2.waitKey(1)

(-65.25, 0.0, 0.03125)
-65.25
-65.25
-64.9992392322845
-65.25
-56.524089433504734
-65.25
-65.25
-65.25
-55.72538591691986
-60.719756059712935
-65.25
-60.94595318088602
-65.25
-65.25
-65.25
-64.45715444715213
-60.61929670516469
-65.25
-65.25
-65.25
-65.25
-64.88449844396416
-65.25
-63.34991635474905
-65.25
-65.25
-65.25
-65.25
-65.25
-59.50618758739994
-55.93000075994637
-54.138109138562854
-62.62043721910588
-65.25
-65.25
-55.44349219587559
-65.25
-65.25
-54.29225581192602
-60.12532726566995
-65.25
-55.46883945131128
-65.25
-65.25
-56.8884533487186
-65.25
-65.25
-65.25
-55.93152340904727
-57.65458653841004
-65.25
-51.90284657392014
-54.72171590212723
-65.25
-53.35708331008581
-62.10958283428126
-52.6143642282306
-54.42313569582471
-65.25
-65.25
-55.43753221981494
-65.25
-65.25
-53.295709608149245
-65.25
-65.25
-60.96755614461711
-53.29162338335413
-62.274514983938865
-54.856445917907415
-65.25
-65.25
-65.25
-65.25
-65.25
-65.25
-65.25
-60.743020618568806
-61.17712687660158
-59.07437787

KeyboardInterrupt: 